# Retrieval and Search

In this tutorial we will put our knowledge database into a search engine which will be used to feed our LLM

In [1]:
import os
import json

from minsearch import Index

In [2]:
documents_file = os.path.join(os.path.dirname(os.getcwd()), 'documents.json')

In [3]:
with open(documents_file, 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
q = 'The course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [64]:
boost = {
    "question":3.0,
    "section:":0.5
}
results = index.search(
    query=q,
    filter_dict={'course':"data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
)

In [65]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Generatiion

In this section we will pass the documents returned by our search engine to our LLM and use it to generate and answer

In [18]:
from dotenv import load_dotenv
from openai import OpenAI
import vertexai
from vertexai.generative_models import GenerativeModel

load_dotenv()

PROJECT_ID = os.getenv('PROJECT_ID')
REGION = os.getenv('REGION')

In [19]:
# Course version (Open AI)
client  = OpenAI()
response = client.chat.completions.create(
    model='gpt4o',
    messages=[{"role":"user", "content":q}]
)


NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt4o` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [20]:
# vertex
vertexai.init(
    project = PROJECT_ID,
    location = REGION
)
vertex_llm = GenerativeModel("gemini-1.5-pro-001")

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0,
    "top_p": 0.95,
}

responses = vertex_llm.generate_content(
    list(q),
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    print(response.text, end="")

c:\Users\User\Documents\LLM_zoomcamp\workshop\llm-zoomcamp-3.10\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Please provide me with more context! I need more information to answer your question.  To figure out if you can still enroll, tell me:

* **What course are you interested in?**  (Give me the name or a brief description)
* **Where is the course offered?** (Is it a school, university, online platform, etc.?)

Once I have this information, I can help you find out if it's still possible to enroll. 😊 


In [66]:
context = ""

for doc in results:
    context += f"section: {doc['question']}\nquestion: {doc['question']}\nanswer: {doc['text']} \n \n"
    

In [67]:
print(context)

section: Course - Can I still join the course after the start date?
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 
 
section: Course - Can I follow the course after it finishes?
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project. 
 
section: Course - When will the course start?
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will

In [31]:
prompt_template = [
    """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

""",
f"""
QUESTION: {q}
""",
f"""
CONTENT: {context}
"""
]

In [36]:
prompt = [element.strip() for element in prompt_template]

In [37]:
for element in prompt:
    print(element)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.
QUESTION: The course has already started, can I still enroll?
CONTENT: section: Course - Can I still join the course after the start date?
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 
 
section: Course - Can I follow the course after it finishes?
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. 

In [38]:
responses = vertex_llm.generate_content(
    prompt,
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    print(response.text, end="")

Yes, you can still submit homework assignments even if you don't register. However, keep in mind that there are deadlines for final projects. 


## Cleaning up the RAG

In [70]:
def search(query):
    boost = {
        "question":3.0,
        "section:":0.5
    }

    results = index.search(
        query=query,
        filter_dict={'course':"data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=10
    )

    context = ""

    for doc in results:
        context += f"section: {doc['question']}\nquestion: {doc['question']}\nanswer: {doc['text']} \n \n"

    return context

In [71]:
def build_prompt(query, search_results):
    prompt_template = [
    """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

""",
f"""
QUESTION: {query}
""",
f"""
CONTENT: {search_results}
"""
]

    prompt = [element.strip() for element in prompt_template]

    return prompt

In [72]:
vertexai.init(
    project = PROJECT_ID,
    location = REGION
)
def llm(prompt):
    
    vertex_llm = GenerativeModel("gemini-1.5-pro-001")

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 0,
        "top_p": 0.95,
    }

    responses = vertex_llm.generate_content(
        prompt,
        generation_config=generation_config,
        stream=True,
    )

    for response in responses:
        print(response.text, end="")

In [75]:
query = 'Can I join after the course has started?'
search_results = search(query)
prompt = build_prompt(query, search_results)
llm(prompt)

Yes, you can submit homework even without registering for the course. However, keep in mind the deadlines for final projects. 


In [76]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    llm(prompt)

In [77]:
rag('the course has already started, can I still join?')

Yes, you can join the course even if it has already started and submit the homework. However, keep in mind the deadlines for final projects. 
